In [ ]:
# necessary package installations -- this cell will autorestart the notebook (ie kill it, you will see notifications for the session crashing, but that is fine)
! pip install --upgrade opengpt
! pip install accelerate
! pip install pandas
! pip install datasets
! pip install pickle
! pip install transformers

import os
os.kill(os.getpid(), 9)

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.2.0
    Uninstalling fsspec-2022.2.0:
      Successfully uninstalled fsspec-2022.2.0


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, pipeline
import pickle
import pandas as pd
import datasets


from opengpt.config import Config
from opengpt.model_utils import add_tokens_to_model_and_tokenizer
from opengpt.dataset_utils import create_labels, pack_examples
from opengpt.data_collator import DataCollatorWithPadding

# switches runtime to GPU
!nvcc --version

In [3]:
# loads the config
config = Config(yaml_path='./example_train_config.yaml')

# this config can be used as a template
config.train.to_dict()

{'model': 'olm/olm-gpt2-oct-2022',
 'datasets': ['../data/example_project_data/prepared_generated_data_for_example_project.csv',
  '../data/nhs_uk_full/prepared_generated_data_for_nhs_uk_qa.csv',
  '../data/nhs_uk_full/prepared_generated_data_for_nhs_uk_conversations.csv',
  '../data/medical_tasks_gpt4/prepared_generated_data_for_medical_tasks.csv'],
 'ignore_index': -100,
 'max_seq_len': 512,
 'packing_type': 'partial',
 'shuffle_dataset': True,
 'hf_training_arguments': {'output_dir': '../data/results/',
  'gradient_accumulation_steps': 16,
  'per_device_eval_batch_size': 1,
  'per_device_train_batch_size': 1,
  'load_best_model_at_end': False,
  'learning_rate': 2e-05,
  'weight_decay': 0.1,
  'adam_beta1': 0.9,
  'adam_beta2': 0.95,
  'adam_epsilon': 1e-07,
  'max_grad_norm': 1,
  'num_train_epochs': 1,
  'lr_scheduler_type': 'cosine',
  'warmup_ratio': 0.03,
  'logging_strategy': 'steps',
  'logging_steps': 100,
  'save_strategy': 'steps',
  'save_steps': 30000,
  'seed': 11,
  'o

In [4]:
# loads the GPT-2 model and tokenizer
model = AutoModelForCausalLM.from_pretrained(config.train.model)
tokenizer = AutoTokenizer.from_pretrained(config.train.model)
tokenizer.model_max_length = config.train.max_seq_len

In [5]:
# expands tokenizer/model with the necessary special tokens for conversational LLMs
add_tokens_to_model_and_tokenizer(config, tokenizer, model)

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50270. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [6]:
# defines the csv datasets to be used
config.train.datasets = ['./prepared_generated_data_for_nhs_uk_qa.csv',
                         './prepared_generated_data_for_nhs_uk_conversations.csv',
                         './prepared_generated_data_for_medical_tasks.csv']

In [7]:
# loads datasets and shuffles if needed
train_dataset = datasets.Dataset.from_csv(config.train.datasets)
if config.train.shuffle_dataset:
    train_dataset = train_dataset.shuffle()
    print("Shuffling dataset!")

Shuffling dataset!


In [8]:
# checks the size of the dataset and columns
train_dataset

Dataset({
    features: ['text', 'raw_data_id'],
    num_rows: 31708
})

In [9]:
# text part used to train model with special tokens
train_dataset[0]['text']

"<|user|> Propose a brief plan for a pilot study to test the effectiveness of a new drug in treating Parkinson's disease. <|eos|> <|ai|> The pilot study will include the following steps:\n1. Obtain ethical approval from the institutional review board (IRB) and gain consent from patients.\n2. Recruit a small sample of participants diagnosed with Parkinson's disease, comprising a balance of genders and ages.\n3. Randomly assign participants to treatment and control groups, ensuring equal numbers in each group.\n4. Administer the new drug to the treatment group and a placebo to the control group, maintaining a double-blind protocol.\n5. Monitor patients using standardized clinical scales to assess symptoms and evaluate motor functions, such as the Unified Parkinson's Disease Rating Scale (UPDRS).\n6. Compare changes in symptom scores over time between the treatment and control groups to discern the drug's effectiveness.\n7. Record any potential side effects or adverse events during the st

In [10]:
# removes everything but text
to_remove = list(train_dataset.column_names)
to_remove.remove('text')
train_dataset = train_dataset.remove_columns(to_remove)

In [12]:
# ignores max_seq_len warning, it is handled by the packer or data_collator
train_dataset = train_dataset.map(
    lambda examples: tokenizer(examples['text'], add_special_tokens=False),
    batched=True,
    num_proc=1,
    remove_columns=["text"])
# creates labels for supervised training (meaning we do not train on questions, but only on answers)
train_dataset = train_dataset.map(
    lambda examples: create_labels(examples, config, tokenizer),
    batched=True,
    batch_size=1000,
    num_proc=1,
)
# we only do packing for the train set
train_dataset = train_dataset.map(
    lambda examples: pack_examples(examples, config.train.max_seq_len, packing_type=config.train.packing_type),
    batched=True,
    batch_size=1000,
    num_proc=1,
)

Map:   0%|          | 0/31708 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (886 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/31708 [00:00<?, ? examples/s]

Map:   0%|          | 0/31708 [00:00<?, ? examples/s]

In [13]:
# checks the new train_dataset (take note of how the labels look). The USER (Question) part of the input should have a label of -100,
# and the AI part (Answer) should have labels equal to input_ids
for i in range(100):
    print(train_dataset[0]['input_ids'][i], train_dataset[0]['labels'][i], train_dataset[0]['attention_mask'][i])

50265 -100 1
980 -100 1
2757 -100 1
225 -100 1
69 -100 1
5090 -100 1
1095 -100 1
326 -100 1
225 -100 1
69 -100 1
9093 -100 1
2621 -100 1
286 -100 1
1735 -100 1
225 -100 1
88 -100 1
76 -100 1
73 -100 1
12035 -100 1
292 -100 1
225 -100 1
69 -100 1
631 -100 1
4002 -100 1
291 -100 1
12422 -100 1
29711 -100 1
526 -100 1
4447 -100 1
18 -100 1
225 -100 1
50267 -100 1
225 -100 1
50266 50266 1
409 409 1
9093 9093 1
2621 2621 1
459 459 1
1706 1706 1
225 225 1
88 88 1
76 76 1
73 73 1
1737 1737 1
3925 3925 1
30 30 1
203 203 1
21 21 1
18 18 1
5723 5723 1
876 876 1
14017 14017 1
8569 8569 1
423 423 1
225 225 1
88 88 1
76 76 1
73 73 1
15462 15462 1
2072 2072 1
2928 2928 1
396 396 1
6393 6393 1
38 38 1
13 13 1
290 290 1
4715 4715 1
12258 12258 1
423 423 1
3533 3533 1
18 18 1
203 203 1
22 22 1
18 18 1
21550 21550 1
225 225 1
69 69 1
1240 1240 1
7588 7588 1
292 292 1
5579 5579 1
13559 13559 1
346 346 1
29711 29711 1
526 526 1
4447 4447 1
16 16 1
22085 22085 1
225 225 1
69 69 1
4610 4610 1
292 292 1
4317

In [14]:
# loads the HF training arguments and make the data collator, you can try increasing the LR or the number of epochs - it could improve the performance of GPT-2.
# check HF Hub for better models with better results
training_args = TrainingArguments(**config.train.hf_training_arguments.to_dict())
dc = DataCollatorWithPadding(tokenizer.pad_token_id, config.train.ignore_index, max_seq_len=config.train.max_seq_len)

# uses HF trainer to train our models
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=None,
    data_collator=dc,
)

In [15]:
# runs training, ignore AdamW warnings
trainer.train()

C:\Users\jhuff19\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,1.613900
200,1.439000
300,1.399000
400,1.373300
500,1.341900
600,1.338800
700,1.340300


TrainOutput(global_step=737, training_loss=1.402439992172262, metrics={'train_runtime': 29127.0495, 'train_samples_per_second': 0.405, 'train_steps_per_second': 0.025, 'total_flos': 2506221310464000.0, 'train_loss': 1.402439992172262, 'epoch': 1.0})

# Testing:

In [17]:
gen = pipeline(model=model, tokenizer=tokenizer, task='text-generation', device=model.device)

In [18]:
t = "<|user|> What is diabetes? <|eos|> <|ai|>" # The format with special tokens is required, because of training
print(gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text'])

C:\Users\jhuff19\anaconda3\lib\site-packages\transformers\generation\utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50267 for open-end generation.


<|user|> What is diabetes? <|eos|> <|ai|> Diabetes is a condition where the body's immune system destroys insulin-producing beta cells in the pancreas. This results in a lack of insulin production, which can lead to high blood sugar levels.
References:
- https://www.nhs.uk/conditions/type-1-diabetes/causes/ 


In [24]:
# Let's try some examples we used to test the NHS-LLM.  Obviously the result is not the best, what is good is the reference is correct - but the explanation makes no sense
t = "<|user|> What is vitamin d3 and should I take it? <|eos|> <|ai|>"
print(gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50267 for open-end generation.


<|user|> What is vitamin d3 and should I take it? <|eos|> <|ai|> Vitamin d3 is a vitamin that helps to protect your skin from damage caused by sun damage, such as sunburn. It is recommended to take it as a supplement to your daily diet.
References:
- https://www.nhs.uk/conditions/vitamins-and-minerals/vitamin-d/ 


In [21]:
# A bit better, it even correctly expanded the abbreviation for hypertension
t = "<|user|> What is HTN? <|eos|> <|ai|>"
print(gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50267 for open-end generation.


<|user|> What is HTN? <|eos|> <|ai|> HTN is a rare genetic disorder that affects the way the brain works. It's caused by a faulty gene that causes the brain to produce a protein called a protein called a protein called a protein kinase.
References:
- https://www.nhs.uk/conditions/neurofibromatosis-type-1/causes/ 


In [23]:
# Let us try a question that has nothing to do with healthcare - this proves that the model is generalising a bit (but the reference makes no sense)
t = "<|user|> What is the capital of France? <|eos|> <|ai|>"
print(gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50267 for open-end generation.


<|user|> What is the capital of France? <|eos|> <|ai|> The capital of France is Paris.
References:
- https://www.nhs.uk/conditions/social-care-and-support-guide/care-services-equipment-and-care-homes/ 


In [ ]:
# And a couple more healthcare quesetions, e.g. an MCQ (The correct answer is A)
t = "<|user|> Choose the correct statement about laparoscopic liver resection efficacy from the following options: a) there is no difference in the overall patient survival rate or disease-free survival rate between laparoscopic liver resection and open resection, b) laparoscopic liver resection has a higher patient survival rate than open resection, c) laparoscopic liver resection has a lower disease-free survival rate than open resection. <|eos|> <|ai|>"
print(gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50267 for open-end generation.


<|user|> Choose the correct statement about laparoscopic liver resection efficacy from the following options: a) there is no difference in the overall patient survival rate or disease-free survival rate between laparoscopic liver resection and open resection, b) laparoscopic liver resection has a higher patient survival rate than open resection, c) laparoscopic liver resection has a lower disease-free survival rate than open resection. <|eos|> <|ai|> The correct statement about laparoscopic liver resection efficacy is that there is no difference in the overall patient


In [25]:
t = "<|user|> I had a high fever for the past 3 days, what should i do? <|eos|> <|ai|>"
print(gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50267 for open-end generation.


<|user|> I had a high fever for the past 3 days, what should i do? <|eos|> <|ai|> If you have a high fever for the past 3 days, you should see a GP. They may recommend a blood test to check for a high temperature. If the high temperature is not present, you may need to take a blood test to check for a high fever.
References:
- https://www.nhs.uk/conditions/high-fever/ 


In [28]:
t = "<|user|> My lower abdomen is hurting very badly on my right side. What should I do? <|eos|> <|ai|>"
print(gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50267 for open-end generation.


<|user|> My lower abdomen is hurting very badly on my right side. What should I do? <|eos|> <|ai|> I suggest you see a GP as soon as possible. They may suggest a tummy tuck or surgery to remove the tummy.
References:
- https://www.nhs.uk/conditions/pregnancy-and-baby/ 


In [ ]:
# Note that we can also continue the conversation (I'm doing it manually here, but can be easily made into a simple chatbot)
t = """<|user|> I had a high fever for the past 3 days, what should i do? <|eos|> <|ai|> I'm sorry to hear that. I understand your concern. Have you been feeling any other symptoms?
References:
- https://www.nhs.uk/conditions/coronavirus-covid-19/coronavirus-vaccination/how-to-get-your-first-dose-for-coronavirus/ <|eos|> <|user|> No, only fever. <|eos|> <|ai|>"""
print(gen(t, do_sample=True, max_length=256, temperature=0.2)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50267 for open-end generation.


<|user|> I had a high fever for the past 3 days, what should i do? <|eos|> <|ai|> I'm sorry to hear that. I understand your concern. Have you been feeling any other symptoms?
References:
- https://www.nhs.uk/conditions/coronavirus-covid-19/coronavirus-vaccination/how-to-get-your-first-dose-for-coronavirus/ <|eos|> <|user|> No, only fever. <|eos|> <|ai|> I understand your concern. Have you been feeling any other symptoms?
References:
- https://www.nhs.uk/conditions/coronavirus-covid-19/coronavirus-vaccination/how-to-get-your-first-dose-for-coronavirus/ 


In [ ]:
# Finally, to show that our training works, we will also try to query an untrained model
model_not_trained = AutoModelForCausalLM.from_pretrained(config.train.model)
tokenizer_not_trained = AutoTokenizer.from_pretrained(config.train.model)
gen_not_trained = pipeline(model=model_not_trained, tokenizer=tokenizer_not_trained, task='text-generation', device=model.device)

In [ ]:
t = "What is HTN?" # No special tokens, as this model is not trained.
print(gen_not_trained(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is HTN?
HTN stands for High-Tensile Nerve Network. It is a network of nerve cells that are linked to the brain. It is a network of nerve cells that are connected to the brain. It is important to note that HTN is not a single nerve cell. It is a network of nerve cells that are linked to the brain.
What is the difference between HTN and the other nerve cells?
The difference between HTN and the other nerve cells is that HTN is


In [ ]:
t = "What is vitamin d3 and should I take it?" # No special tokens, as this model is not trained.
print(gen_not_trained(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is vitamin d3 and should I take it?
Vitamin D3 is a vitamin that is found in the skin. It is a vitamin that is essential for the health of the skin. It is also essential for the growth of the skin. It is important to take vitamin D3 if you are looking for a healthy glow.
What is vitamin d2 and what is vitamin d3?
Vitamin D2 is a vitamin that is found in the skin. It is a vitamin that is essential for th


In [ ]:
t = "What is the capital of France?" # Let's try a general question
print(gen_not_trained(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is the capital of France?
The capital of France is Paris. It is the capital of France and the second largest city in the world. It is also the capital of the French overseas department of Guadeloupe.
What is the capital of the United Kingdom?
The capital of the United Kingdom is London. It is the capital of the United Kingdom and the second largest city in the world. It is the capital
